In [1]:
facts= "/home/liorkob/thesis/lcp/processed_verdicts_with_gpt.csv"

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

def load_text_for_verdict(df, verdict, part):
    # Filter the DataFrame for the specific verdict
    verdict_df = df[df['verdict'] == verdict]
    
    # Extract the text column based on the 'part' (can be 'extracted_facts' or 'extracted_gpt_facts')
    text = " ".join(verdict_df[part].values)
    
    # Slice the text based on start_part and end_part indices (if they are numerical)
    return text

# Load CSV
df = pd.read_csv("/home/liorkob/thesis/lcp/processed_verdicts_with_gpt.csv")  

# Parameters
part = 'extracted_gpt_facts'  # Define the part to extract features from

# Load text for each verdict (A, B, C)
text_a = load_text_for_verdict(df, 'ת"פ 16420-10-16', part)
text_b = load_text_for_verdict(df, 'ת"פ 40313-09-15', part)
text_c = load_text_for_verdict(df, 'ת"פ 1995-03-17', part)
print(text_a)

הנאשם הורשע על פי הכרעת דין בעבירות של חבלה בכוונה מחמירה לפי סעיף 329(א)(1)+(2) לחוק העונשין, התשל"ז-1977 והחזקת ונשיאת נשק ותחמושת לפי סעיפים 144(א) רישא וסיפא + 144(ב) רישא וסיפא לחוק העונשין. כאמור בהכרעת הדין על רקע סכסוך משפחתי הגיע הנאשם מול ביתו של המתלונן כשהוא ואחיו שיפצו קיר מחוץ לבית, כאשר הוא הוסע במכונית על ידי אחר וירה עליהם באמצעות נשק חם- תת מקלע. כדור אחד פגע ברגלו של המתלונן. המתלונן אושפז למשך חמישה ימים ועבר מספר ניתוחים.


In [9]:


# TF-IDF Vectorization
vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf_matrix = vectorizer.fit_transform([text_a, text_b, text_c])

# Get feature names
features = vectorizer.get_feature_names_out()

# Extract important tokens
a_b_common = set(features[tfidf_matrix[0].toarray().argsort()[0][-10:]]) & \
             set(features[tfidf_matrix[1].toarray().argsort()[0][-10:]])
a_c_diff = set(features[tfidf_matrix[0].toarray().argsort()[0][-10:]]) - \
           set(features[tfidf_matrix[2].toarray().argsort()[0][-10:]])

# Significant features
significant_features = a_b_common - a_c_diff
print("Significant Features:", significant_features)


Significant Features: set()


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import matplotlib.pyplot as plt
import seaborn as sns

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("avichr/Legal-heBERT")
model = AutoModel.from_pretrained("avichr/Legal-heBERT")

# Tokenize inputs
inputs = tokenizer(text_a, text_b, return_tensors='pt', truncation=True, padding=True)

# Forward pass with output_attentions=True
outputs = model(**inputs, output_attentions=True)

# Get attention weights from the last layer
attention_weights = outputs.attentions[-1]  # shape: (batch_size, num_heads, seq_len, seq_len)
print("Attention weights shape:", attention_weights.shape)

# Average over heads
avg_attention = attention_weights[0].mean(dim=0).detach().numpy()

# Convert tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Plot heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(avg_attention, xticklabels=tokens, yticklabels=tokens, cmap='viridis')
plt.title("Average Attention - Last Layer")
plt.show()



Some weights of BertModel were not initialized from the model checkpoint at avichr/Legal-heBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'text_a' is not defined

: 

In [ ]:
import torch

# Get average attention across all heads
avg_attention = attention_weights[0].mean(dim=0)  # (seq_len, seq_len)

# Sum attention received by each token (columns)
attention_to_token = avg_attention.sum(dim=0)

# Get top tokens by attention score
topk = torch.topk(attention_to_token, k=10)
top_token_indices = topk.indices.tolist()
top_tokens = [tokens[i] for i in top_token_indices]

import re

def is_informative(token):
    return token not in ['[CLS]', '[SEP]'] and not re.fullmatch(r'[.,!?״׳:;…"\-(){}\[\]]+', token)

# Filter top tokens
filtered_top_tokens = [t for t in top_tokens if is_informative(t)]
print("Filtered top-attended tokens:", filtered_top_tokens)


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f0538d3a4f0>>
Traceback (most recent call last):
  File "/home/liorkob/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


RuntimeError: duplicate registrations for aten.linspace.Tensor_Tensor